In [ ]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [ ]:
train=pd.read_csv("../input/Train.csv")
test=pd.read_csv("../input/Test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])

In [ ]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

In [ ]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

In [ ]:
for x in range(121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [ ]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

In [ ]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

In [ ]:
data.drop(features,1,inplace=True)

In [ ]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [ ]:
del data  
gc.collect()

In [ ]:
test.drop("target",axis  = 1,inplace  = True)

In [ ]:
train.shape,test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [ ]:
lab.fit(train["location"])

In [ ]:
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [ ]:
train.target.min(),train.target.max()

In [ ]:
!pip install catboost

In [ ]:
X=train.drop(["ID",'target'],axis =1)
y = train.target

In [ ]:
test_id  = test['ID']

In [ ]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=2)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(   n_estimators=1200,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

In [ ]:
np.mean(errcb2)

In [ ]:
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [ ]:
sub.target.min(),sub.target.max()

# Ensemble

In [ ]:
a=pd.read_csv('/kaggle/working/c1.csv')

In [ ]:
b=pd.read_csv('/kaggle/working/c2.csv')

In [ ]:
a.head(2)

In [ ]:
a['target']=a['target']*0.73 +b['target']*0.27

In [ ]:
a.to_csv('ensem.csv',index=False)